In [1]:
using CairoMakie, Dierckx, Optim, LinearAlgebra, QuantEcon

In [2]:
pars = (;α = 0.33, # Capital share
        β = 0.9, # Discount factor
        A = 10.0, # TFP
        γ = 2.0, # Risk aversion
        δ = 0.1, # Depreciation rate
        nk = 31, # Number of capital gridpoints
        θ = 4, # Grid expansion parameter
        lb = 1e-2, # Lower bound of capital grid
        ub = 1000.0, # Upper bound of capital grid
        nz = 19, # Number of shock gridpoints
        ρ = 0.9, # Persistence of AR(1) process
        μ = 0.0, # Mean of AR(1) process
        σ = 0.007, # Var of AR(1) process
        toler = 1e-6, # Tolerance
        maxiter = 10000) # Maximum number of iterations

(α = 0.33, β = 0.9, A = 10.0, γ = 2.0, δ = 0.1, nk = 31, θ = 4, lb = 0.01, ub = 1000.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.007, toler = 1.0e-6, maxiter = 10000)

In [3]:
function utility(c)
    if pars.γ == 1.0
        return log(c)
    else
        return (c^(1-pars.γ)) / (1-pars.γ)
    end
end 

function ar1(pars)
    (; μ, ρ, σ, nz) = pars
    mc = rouwenhorst(nz, μ, ρ, σ)
    Π, Zvals = mc.p, exp.(mc.state_values)
    return Π, Zvals
end

function exp_grid(pars)
    (; nk, θ, lb, ub) = pars
    grid = LinRange(1e-4, 1.0, nk)
    expgrid = lb .+ (ub .- lb) .* grid.^θ
    return expgrid
end

function interpolate(grid, vals, pars)
    spline = Spline1D(grid, vals, k = 1, bc = "extrapolate")
    return spline
end

function production(k, z ,pars)
    (; α, δ, A) = pars
    return A * z * (k ^ α) + (1.0 - δ) * k
end

function initial_guess(grid, pars)
    (; nk, nz) = pars
    v_out = zeros(nk, nz)
    for j in 1:nz
        v_out[:,j] = j * utility.(grid)
    end
    return v_out
end

function resource_grid(kgrid, zgrid, pars)
    (; nk, nz) = pars
    Ygrid = zeros(nk, nz)
    for i in 1:nk
        for j in 1:nz
            Ygrid[i,j] = production(kgrid[i], zgrid[j], pars)
        end
    end
    return Ygrid
end

resource_grid (generic function with 1 method)

In [4]:
function egm(pars)
    (; γ, nk, nz, toler, maxiter, β) = pars

    Π, Zvals = ar1(pars)
    Kvals = exp_grid(pars)

    v1 = initial_guess(exp_grid(pars), pars)
    v2 = zeros(nk, nz)
    v3 = zeros(nk, nz)
    v4 = zeros(nk, nz)

    c = zeros(nk, nz)

    derivatives = zeros(nk, nz)

    Yvals = resource_grid(Kvals, Zvals, pars)
    Ystar = zeros(nk, nz)

    Kinterpolators = Vector{Spline1D}(undef, nz)
    Yinterpolators = Vector{Spline1D}(undef, nz)

    error = toler + 1
    iter = 0
    if iter == 0
        println("Iterating...")
    end
    while ((error > toler) && (iter < maxiter))
        Kinterpolators = [interpolate(Kvals, v1[:,j], pars) for j in 1:nz]
        for (j, interpolator) in enumerate(Kinterpolators)
            derivatives[:,j] = Dierckx.derivative(interpolator, Kvals)
        end
        c = (derivatives).^(-1/γ)
        Ystar = c .+ Kvals
        v2 = utility.(c) .+ v1
        Yinterpolators = [interpolate(Ystar[:,j], v2[:,j], pars) for j in 1:nz]
        for (j, interpolator) in enumerate(Yinterpolators)
            v3[:,j] = interpolator(Yvals[:,j])
        end
        v4 = β * (v3 * Π')
        #for i in 1:nk
        #    for j in 1:nz
        #        v4[i,j] = β * dot(v3[i,:], Π[j,:])
        #    end
        #end
        error = maximum(abs.(v4 - v1) ./ (1.0 .+ abs.(v1)))
        if iter % 50 == 0
            println("--------------------")
            println("Iteration: $iter, Error: $error")
        end
        v1 .= v4
        iter += 1
    end
    println("--------------------")
    println("Converged in $iter iterations")
    println("--------------------")
    return v1
end

@time begin
    test = egm(pars) 
end

Yvals = [0.05738827789793602 0.0829597969349837 0.12204497287965202 0.18178530259617898 0.27309631523411004 0.41266201680501696 0.62598332923221 0.9520375232315906 1.450399999136898 2.21212968077056 3.3764069607316785 5.155964220108083 7.875955491865114 12.033367387128868 18.387826626152677 28.100396185694297 42.94572037466908 65.63628119122 100.31801286751231; 0.060428561439901586 0.08701274581986682 0.12764574546025131 0.1897518697700031 0.2846789170201593 0.42977160151437505 0.6515407147815467 0.9905070780461837 1.508605354817631 2.300500520486264 3.510884691839121 5.360914728358601 8.188621184730046 12.510671986914708 19.116776059200813 29.21397597915919 44.64719427363353 68.23633074839991 104.291504012189; 0.09630585690420689 0.1329963966330698 0.18907660832209028 0.27479325807230653 0.4058081708751367 0.6060599005770416 0.9121376927208573 1.3799669343613192 2.0950276311891782 3.1879729938069756 4.858501902705061 7.411847139268477 11.314546375632801 17.27968618963554 26.3971945717

Excessive output truncated after 533543 bytes.

31×19 Matrix{Float64}:
 -1.37413   -1.37413   -1.37413   …  -1.37413   -1.37413   -1.37413
 -1.34043   -1.34043   -1.34043      -1.34043   -1.34043   -1.34043
 -1.09009   -1.09009   -1.09009      -1.09009   -1.09009   -1.09009
 -0.838229  -0.838229  -0.838229     -0.838229  -0.838229  -0.838229
 -0.675478  -0.675478  -0.675478     -0.675478  -0.675478  -0.675478
 -0.567142  -0.567142  -0.567142  …  -0.567142  -0.567142  -0.567142
 -0.493551  -0.493551  -0.493551     -0.493551  -0.493551  -0.493551
 -0.438422  -0.438422  -0.438422     -0.438422  -0.438422  -0.438422
 -0.394129  -0.394129  -0.394129     -0.394129  -0.394129  -0.394129
 -0.357267  -0.357267  -0.357267     -0.357267  -0.357267  -0.357267
  ⋮                               ⋱                        
 -0.14949   -0.14949   -0.14949      -0.14949   -0.14949   -0.14949
 -0.141684  -0.141684  -0.141684     -0.141684  -0.141684  -0.141684
 -0.134412  -0.134412  -0.134412     -0.134412  -0.134412  -0.134412
 -0.12762   -0.12762   -